Question 1: Use any GAN of your choice (preferably DCGAN) to generate images from noise. Perform the
following experiments.
A. Use the CIFAR 10 database to learn the GAN network. Generate images once the learning is complete.
B. Plot generator and discriminator losses and show how can you ascertain the convergence of the GAN
training process.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the generator and discriminator networks
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),

            nn.ConvTranspose2d(ngf * 2, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(ndf * 4, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)


# Set hyperparameters
nz = 100  # Size of the latent vector
ngf = 64  # Generator feature maps
ndf = 64  # Discriminator feature maps
nc = 3    # Number of channels in the images

# Create generator and discriminator instances
netG = Generator(nz, ngf, nc).to(device)
netD = Discriminator(nc, ndf).to(device)

# Define loss function and optimizers
criterion = nn.BCELoss()
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))

# Set up data loader for CIFAR-10
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

dataset = datasets.CIFAR10(root="./data", download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True, num_workers=4)

# Training the GAN
num_epochs = 10
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Update discriminator
         netD.zero_grad()
    real_images = data[0].to(device)
    batch_size = real_images.size(0)
    label = torch.full((batch_size,), 1, device=device)

    output = netD(real_images).view(-1)
    errD_real = criterion(output, label[:batch_size])  # Fix here
    errD_real.backward()

    noise = torch.randn(batch_size, nz, 1, 1, device=device)
    fake_images = netG(noise)
    label.fill_(0)
    output = netD(fake_images.detach()).view(-1)
    errD_fake = criterion(output, label[:batch_size])  # Fix here
    errD_fake.backward()
    optimizerD.step()

    # Update generator
    netG.zero_grad()
    label.fill_(1)
    output = netD(fake_images).view(-1)
    errG = criterion(output, label[:batch_size])  # Fix here
    errG.backward()
    optimizerG.step()

    # Print training statistics
    if i % 100 == 0:
        print(f"[{epoch}/{num_epochs}][{i}/{len(dataloader)}] Loss_D: {errD.item()} Loss_G: {errG.item()}")
        # Update generator
        netG.zero_grad()
        label.fill_(1)
        output = netD(fake_images).view(-1)
        errG = criterion(output, label)
        errG.backward()
        optimizerG.step()

        # Print training statistics
        if i % 100 == 0:
            print(f"[{epoch}/{num_epochs}][{i}/{len(dataloader)}] Loss_D: {errD.item()} Loss_G: {errG.item()}")

# Save the trained models
torch.save(netG.state_dict(), "generator.pth")
torch.save(netD.state_dict(), "discriminator.pth")

# Generate images using the trained generator
with torch.no_grad():
    fixed_noise = torch.randn(64, nz, 1, 1, device=device)
    fake_images = netG(fixed_noise).detach().cpu()

# Plot generated images
fig, axes = plt.subplots(8, 8, figsize=(12, 12))
for i, ax in enumerate(axes.flatten()):
    ax.imshow(np.transpose(fake_images[i], (1, 2, 0)))
    ax.axis('off')
plt.show()





In [ ]:
# Lists to store generator and discriminator losses during training
G_losses = []
D_losses = []

# Training loop (inside the main loop)
for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # ... (same training loop as before)

        # Append losses to lists
        G_losses.append(errG.item())
        D_losses.append((errD_real + errD_fake).item())

# Plot the losses
plt.figure(figsize=(10, 5))
plt.plot(G_losses, label="Generator Loss")
plt.plot(D_losses, label="Discriminator Loss")
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.legend()
plt.show()


Question 2: Fine-tuning Take a ResNet50 model and the database to be used for this question is CIFAR-10.
Remove its classification layer and place a 2-layer neural network followed by a Softmax layer. Calculate
classification accuracy on a train set, test set, and plot accuracies over epochs when:
A. The complete network is trained from scratch (i.e, random weights)
B. A pre-trained ResNet50 on ImageNet weights is used and only the neural network layers are trained
(i.e, weights of layers of ResNet50 are kept frozen and unchanged)
C. A pre-trained ResNet50 on ImageNet weights is used and all the layers are adapted (i.e, weights of
layers of ResNet50 are also updated now)
D. Using a ResNet50 model for CIFAR-10, propose your own domain adaptation algorithm. To get full
credit for this part, the accuracy on the test set should be more than what was reported in part 3. You
may build upon part (3) to propose your own algorithm. Explain why your proposed algorithm is
working better. You may use any training data as long as it involves using other datasets (on which
you'll adapt CIFAR-10).

Certainly! Let's break down each part of the question and provide code snippets for each scenario:

### A. Train the Complete Network from Scratch:

```python
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
])

train_dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# Define the model
class CustomResNet50(nn.Module):
    def __init__(self, num_classes=10):
        super(CustomResNet50, self).__init__()
        resnet = models.resnet50(pretrained=False)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Remove the last classification layer
        self.fc_layers = nn.Sequential(
            nn.Linear(2048, 512),
            nn.ReLU(),
            nn.Linear(512, num_classes),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc_layers(x)
        return x

# Initialize and train the model
model_scratch = CustomResNet50().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_scratch = optim.SGD(model_scratch.parameters(), lr=0.001, momentum=0.9)

def train(model, train_loader, criterion, optimizer, num_epochs=10):
    model.train()
    train_accuracies = []

    for epoch in range(num_epochs):
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        accuracy = correct / total
        train_accuracies.append(accuracy)

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {accuracy:.4f}")

    return train_accuracies

# Train the complete network from scratch
train_accuracies_scratch = train(model_scratch, train_loader, criterion, optimizer_scratch)

# Plot accuracies over epochs
plt.plot(train_accuracies_scratch, label='Training Accuracy (Scratch)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Over Epochs (Scratch)')
plt.legend()
plt.show()
```

### B. Fine-tune Only the Neural Network Layers with Pre-trained ResNet50 Weights:

```python
# Load pre-trained ResNet50 model
model_pretrained = models.resnet50(pretrained=True)
model_pretrained = nn.Sequential(*list(model_pretrained.children())[:-1])  # Remove the last classification layer

# Add custom layers
model_pretrained.fc_layers = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Linear(512, 10),  # 10 classes for CIFAR-10
    nn.Softmax(dim=1)
)

# Fine-tune only the neural network layers
model_pretrained.fc_layers.parameters()

# Initialize and train the model
model_pretrained = model_pretrained.to(device)
optimizer_pretrained = optim.SGD(model_pretrained.parameters(), lr=0.001, momentum=0.9)

# Train only the neural network layers
train_accuracies_pretrained = train(model_pretrained, train_loader, criterion, optimizer_pretrained)

# Plot accuracies over epochs
plt.plot(train_accuracies_pretrained, label='Training Accuracy (Pretrained)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Over Epochs (Pretrained)')
plt.legend()
plt.show()
```

### C. Fine-tune All Layers (Including ResNet50) with Pre-trained Weights:

```python
# Load pre-trained ResNet50 model
model_all_layers = models.resnet50(pretrained=True)

# Modify the classification layer
model_all_layers.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Linear(512, 10),  # 10 classes for CIFAR-10
    nn.Softmax(dim=1)
)

# Fine-tune all layers
model_all_layers.parameters()

# Initialize and train the model
model_all_layers = model_all_layers.to(device)
optimizer_all_layers = optim.SGD(model_all_layers.parameters(), lr=0.001, momentum=0.9)

# Train all layers
train_accuracies_all_layers = train(model_all_layers, train_loader, criterion, optimizer_all_layers)

# Plot accuracies over epochs
plt.plot(train_accuracies_all_layers, label='Training Accuracy (All Layers)')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy Over Epochs (All Layers)')
plt.legend()
plt.show()
```

### D. Domain Adaptation Algorithm:

For domain adaptation, you may use techniques such as domain adversarial training or self-training with pseudo-labels on other datasets. The key is to leverage data from a source domain (e.g., ImageNet) to improve performance on the target domain (CIFAR-10). Here's a simplified example using domain adversarial training:

```python
# Import necessary libraries
from torch.autograd import Function

# Define the domain discriminator
class DomainDiscriminator(nn.Module):
    def __init__(self):
        super(DomainDiscriminator, self).__init__()
        self.fc1 = nn.Linear(2048, 1024)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(1024, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize models and optimizers
model_domain_adaptation = models.resnet50(pretrained=True)
model_domain_adaptation.fc = nn.Sequential(
    nn.Linear(2048, 512),
    nn.ReLU(),
    nn.Linear(512, 10),
    nn.Softmax(dim=1)
)

discriminator = DomainDiscriminator()

model_domain_adaptation = model_domain_adaptation.to(device)
discriminator = discriminator.to(device)

optimizer_domain_adaptation = optim.SGD([
    {'params': model_domain_adaptation.parameters()},
    {'params': discriminator.parameters()}
], lr=0.001, momentum=0.9)

# Define the domain adversarial training function
class GradientReverse

(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return grad_output.neg() * ctx.alpha, None

def grad_reverse(x, alpha):
    return GradientReverse.apply(x, alpha)

# Domain adversarial training loop
def train_domain_adaptation(model, discriminator, dataloader_source, dataloader_target, criterion, optimizer, num_epochs=10, alpha=0.01):
    model.train()
    discriminator.train()
    train_accuracies = []

    for epoch in range(num_epochs):
        for (source_inputs, source_labels), (target_inputs, _) in zip(dataloader_source, dataloader_target):
            source_inputs, source_labels = source_inputs.to(device), source_labels.to(device)
            target_inputs = target_inputs.to(device)

            # Source domain classification
            optimizer.zero_grad()
            outputs_source = model(source_inputs)
            loss_source = criterion(outputs_source, source_labels)
            loss_source.backward()

            # Domain adaptation
            optimizer_domain_adaptation.zero_grad()
            features = model_domain_adaptation(source_inputs)
            features = grad_reverse(features, alpha)
            domain_labels_source = torch.ones(source_inputs.size(0), 1).to(device)
            outputs_domain_source = discriminator(features)
            loss_domain_source = criterion(outputs_domain_source, domain_labels_source)

            features = model_domain_adaptation(target_inputs)
            features = grad_reverse(features, alpha)
            domain_labels_target = torch.zeros(target_inputs.size(0), 1).to(device)
            outputs_domain_target = discriminator(features)
            loss_domain_target = criterion(outputs_domain_target, domain_labels_target)

            loss_domain_adaptation = loss_domain_source + loss_domain_target
            loss_domain_adaptation.backward()

            optimizer.step()
            optimizer_domain_adaptation.step()

            # Print training statistics
            if i % 100 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Loss Source: {loss_source:.4f}, Loss Domain Adaptation: {loss_domain_adaptation:.4f}")

        # Evaluate on the target domain
        accuracy_target = evaluate(model, dataloader_target)
        train_accuracies.append(accuracy_target)
        print(f"Epoch {epoch+1}/{num_epochs}, Accuracy on Target Domain: {accuracy_target:.4f}")

    return train_accuracies

# Train the model with domain adaptation
train_accuracies_domain_adaptation = train_domain_adaptation(model_domain_adaptation, discriminator, train_loader, test_loader, criterion, optimizer_domain_adaptation)

# Plot accuracies over epochs
plt.plot(train_accuracies_domain_adaptation, label='Accuracy with Domain Adaptation')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy on Target Domain with Domain Adaptation')
plt.legend()
plt.show()
```

This example uses a simple domain adversarial training approach. You may need to fine-tune and adjust hyperparameters based on the specific characteristics of your source and target domains. Additionally, you may explore other advanced domain adaptation techniques, such as self-training, consistency regularization, or domain adversarial neural networks (DANN).

Question 3: Implement a gan from scratch using Keras to generate celebrity faces from noise using this
data-: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
Use cases found for GAN:
• Super-resolution: increasing the resolution of input images
• Colorise blank and white images
• image inpainting - fill missing blocks in images
• Anime face generation
• font generation
• style transfer
• human face generation
• image to emoji
• GAN for data augmentation
• Face ageing GAN
• front facial view generation from images provided of different sides
• Photo blending- blending 2 images

Sure, I'll provide you with a simple implementation of a Generative Adversarial Network (GAN) using Keras to generate celebrity faces from noise. For this example, we'll focus on generating realistic celebrity faces. You can further modify the architecture and training process to adapt to specific use cases mentioned.

Please note that training a GAN can be computationally expensive and may take a long time. Also, the provided code is a basic example; you may need to experiment with the model architecture, hyperparameters, and other settings to achieve better results.

### Install Required Libraries

Make sure you have Keras and TensorFlow installed:

```bash
pip install keras tensorflow
```

### GAN Implementation for Celebrity Faces Generation

```python
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from keras.optimizers import Adam
from keras import initializers

# Load the CelebA dataset
# Download the dataset from https://www.kaggle.com/jessicali9530/celeba-dataset
# Extract the images into a folder and provide the path below
# For this example, I'm assuming you have images in a 'celeba' folder
# You may need to preprocess the images based on your specific use case
# (e.g., resizing to a fixed size)
images_path = 'path_to_celeba_images_folder'

# Define image dimensions
img_rows, img_cols, channels = 64, 64, 3
img_shape = (img_rows, img_cols, channels)
latent_dim = 100  # Size of the latent space

# Load and preprocess CelebA images
def load_celeba_data(images_path):
    # Load images and normalize to the range [-1, 1]
    images = np.load(images_path)
    images = (images.astype(np.float32) - 127.5) / 127.5
    return images

celeba_images = load_celeba_data(images_path)

# Generator model
def build_generator(latent_dim):
    model = Sequential()

    model.add(Dense(256, input_dim=latent_dim, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))

    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    return model

# Discriminator model
def build_discriminator(img_shape):
    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(1024, kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dense(1, activation='sigmoid'))

    return model

# Build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5), metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Build and compile the combined model
z = Input(shape=(latent_dim,))
img = generator(z)

discriminator.trainable = False
validity = discriminator(img)

combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=Adam(0.0002, 0.5))

# Training the GAN
def train_gan(epochs, batch_size, save_interval):
    half_batch = batch_size // 2

    for epoch in range(epochs):
        # Select a random half batch of images
        idx = np.random.randint(0, celeba_images.shape[0], half_batch)
        imgs = celeba_images[idx]

        # Sample noise and generate a half batch of new images
        noise = np.random.normal(0, 1, (half_batch, latent_dim))
        gen_imgs = generator.predict(noise)

        # Train the discriminator (real classified as ones and generated as zeros)
        d_loss_real = discriminator.train_on_batch(imgs, np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch(gen_imgs, np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator (wants discriminator to mistake images as real)
        noise = np.random.normal(0, 1, (batch_size, latent_dim))
        valid_labels = np.ones((batch_size, 1))

        g_loss = combined.train_on_batch(noise, valid_labels)

        # Plot the progress
        print(f"{epoch} [D loss: {d_loss[0]} | D accuracy: {100 * d_loss[1]}] [G loss: {g_loss}]")

        # Save generated images at specified intervals
        if epoch % save_interval == 0:
            save_generated_images(epoch)

# Function to save generated images
def save_generated_images(epoch):
    r, c = 5, 5  # Grid size
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c, figsize=(10, 10))
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt])
            axs[i, j].axis('off')
            cnt += 1
    plt.savefig(f"gan_generated_image_epoch_{epoch}.png")
    plt.close()

# Train the GAN
epochs = 10000  # You may need more epochs based on your dataset and model complexity
batch_size = 64
save_interval = 1000

train_gan(epochs, batch_size, save_interval)
```

This is a basic GAN architecture for generating celebrity faces using Keras. You can customize the architecture, hyperparameters, and training loop based on your specific use case. Keep in mind that GAN training can be sensitive to hyperparameters, and it may require experimentation to achieve good results.